In [ ]:
from datasets import load_dataset

# Load the MedQA dataset (US version)
dataset = load_dataset("VodLM/medqa", "us")

# Check available splits
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

medqa.py:   0%|          | 0.00/5.45k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/7.69M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/964k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/997k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1272 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'uid', 'question', 'metamap', 'target', 'answers'],
        num_rows: 10178
    })
    validation: Dataset({
        features: ['idx', 'uid', 'question', 'metamap', 'target', 'answers'],
        num_rows: 1272
    })
    test: Dataset({
        features: ['idx', 'uid', 'question', 'metamap', 'target', 'answers'],
        num_rows: 1273
    })
})


In [ ]:
train_data = dataset["train"]
print(train_data[0])  # Print the first data point



import pandas as pd

# Convert dataset to pandas DataFrame
train_data = dataset["train"].to_pandas()
test_data = dataset["test"].to_pandas()
validation_data = dataset["validation"].to_pandas()




{'idx': 0, 'uid': 'train-0', 'question': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'metamap': '23 year old pregnant woman weeks presents burning urination states started 1 day worsening drinking water taking cranberry extract feels well followed by doctor pregnancy temperature 97 36 blood pressure mmHg pulse 80 min respirations min oxygen saturation 98 room air Physical exam notable absence costovertebral angle tenderness gravid uterus following best 

In [ ]:
train_data.head()

,idx,uid,question,metamap,target,answers
0,0,train-0,A 23-year-old pregnant woman at 22 weeks gesta...,23 year old pregnant woman weeks presents burn...,3,"[Ampicillin, Ceftriaxone, Doxycycline, Nitrofu..."
1,1,train-1,A 3-month-old baby died suddenly at night whil...,3 month old baby died night asleep mother died...,0,[Placing the infant in a supine position on a ...
2,2,train-2,A mother brings her 3-week-old infant to the p...,mother week old infant pediatrician's office c...,0,"[Abnormal migration of ventral pancreatic bud,..."
3,3,train-3,A pulmonary autopsy specimen from a 58-year-ol...,pulmonary autopsy specimen 58 year old woman d...,0,"[Thromboembolism, Pulmonary ischemia, Pulmonar..."
4,4,train-4,A 20-year-old woman presents with menorrhagia ...,20 year old woman presents menorrhagia past ye...,3,"[Hemophilia A, Lupus anticoagulant, Protein C ..."


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 42.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

# Load BioGPT for causal language modeling
model_name = "microsoft/BioGPT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(model_name)


# Access the internal BioGptModel
transformer_layers = model.biogpt.layers   # BioGPT uses this structure

# Freeze all but the top 4 transformer blocks
# num_layers = len(transformer_layers)
# for i, layer in enumerate(transformer_layers):
#     if i < num_layers - 4:
#         for param in layer.parameters():
#             param.requires_grad = False

# # Optionally freeze embeddings too
# for param in model.biogpt.embed_tokens.parameters():
#     param.requires_grad = False

# # Unfreeze last 6 layers of the transformer
# for name, param in base_model.named_parameters():
#     if any(f'model.decoder.layers.{i}' in name for i in range(8, 12)):
#         param.requires_grad = True
#     else:
#         param.requires_grad = False

LETTER_MAP = ["A", "B", "C", "D"]

def build_prompt(example):
    question = example["question"]
    options = example["answers"]
    prompt = (
        f"Question: {question}\n"
        "Options:\n"
        f"A: {options[0]}\n"
        f"B: {options[1]}\n"
        f"C: {options[2]}\n"
        f"D: {options[3]}\n"
        "Choose the best option (A, B, C, or D). Answer:"
    )
    return prompt, LETTER_MAP[example["target"]]

class QAGenerationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        prompt, answer = build_prompt(self.data[idx])
        input_text = prompt + " " + answer

        encoding = self.tokenizer(
            input_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Mask all prompt tokens with -100 so loss is only computed for the answer
        input_ids = encoding["input_ids"].squeeze(0)
        attention_mask = encoding["attention_mask"].squeeze(0)
        answer_token_ids = self.tokenizer(answer, add_special_tokens=False)["input_ids"]

        # Label: mask prompt with -100, retain only last tokens
        answer_start = len(input_ids) - len(answer_token_ids)
        labels = torch.full_like(input_ids, -100)
        labels[answer_start:] = torch.tensor(answer_token_ids)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# Convert dataframe to dict
train_examples = train_data.to_dict(orient="records")
val_examples = validation_data.to_dict(orient="records")

train_dataset = QAGenerationDataset(train_examples, tokenizer)
val_dataset = QAGenerationDataset(val_examples, tokenizer)

training_args = TrainingArguments(
    output_dir="./biogpt_qa_generation",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_safetensors=False,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)



config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,1.422400,1.404879
2,1.411600,1.389022
3,1.283800,1.415284
4,0.977900,1.711698
5,0.562300,2.242815


Evaluation Results: {'eval_loss': 1.389021635055542, 'eval_runtime': 10.2577, 'eval_samples_per_second': 124.004, 'eval_steps_per_second': 15.5, 'epoch': 5.0}


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import random

LETTER_MAP = ["A", "B", "C", "D"]

# ------------------ Load Models ------------------
model_name = "microsoft/BioGPT"
finetuned_model = AutoModelForCausalLM.from_pretrained("./biogpt_qa_generation/checkpoint-3185").to("cuda")
zeroshot_model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
encoder = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")

# ------------------ Prompting ------------------
def build_prompt(example):
    return (
        f"Question: {example['question']}\n"
        "Options:\n"
        f"A: {example['answers'][0]}\n"
        f"B: {example['answers'][1]}\n"
        f"C: {example['answers'][2]}\n"
        f"D: {example['answers'][3]}\n"
        "Answer:"
    )

def format_cot_few_shot_prompt(shot_examples, test_example):
    prompt = "You are a helpful assistant. Eliminate incorrect choices step by step. Then answer with the best option text.\n\n"
    for ex in shot_examples:
        prompt += build_prompt(ex) + f"\nExplanation: Based on context, best answer is\nAnswer: {ex['answers'][ex['target']]}\n\n"
    prompt += build_prompt(test_example)
    return prompt

# ------------------ Evaluation Methods ------------------
def predict_by_scoring(model, tokenizer, example):
    prompt = build_prompt(example)
    losses = []
    for option in example['answers']:
        input_text = prompt + " " + option
        enc = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(model.device)
        with torch.no_grad():
            output = model(**enc, labels=enc["input_ids"])
            losses.append(output.loss.item())
    return int(np.argmin(losses))

def predict_by_prompting(model, tokenizer, encoder, example, few_shot_bank):
    prompt = format_cot_few_shot_prompt(few_shot_bank, example)
    enc = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(model.device)
    with torch.no_grad():
        output = model.generate(**enc, max_new_tokens=32)
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    answer_text = generated.split("Answer:")[-1].strip().lower()
    ans_emb = encoder.encode(answer_text, convert_to_tensor=True)
    opt_emb = encoder.encode([a.lower() for a in example['answers']], convert_to_tensor=True)
    sim = util.cos_sim(ans_emb, opt_emb)
    return int(torch.argmax(sim))

def predict_by_finetuned(model, tokenizer, example):
    prompt, answer = build_prompt(example), LETTER_MAP[example['target']]
    input_text = prompt + " " + answer
    enc = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    input_ids = enc['input_ids'].to(model.device)
    attn = enc['attention_mask'].to(model.device)
    with torch.no_grad():
        output = model.generate(input_ids=input_ids, attention_mask=attn, max_new_tokens=1)
    gen = tokenizer.decode(output[0], skip_special_tokens=True)
    for i, opt in enumerate(LETTER_MAP):
        if opt in gen:
            return i
    return -1

# ------------------ Ensemble Prediction ------------------
def ensemble_predict(example, few_shot_bank):
    pred1 = predict_by_finetuned(finetuned_model, tokenizer, example)
    pred2 = predict_by_scoring(zeroshot_model, tokenizer, example)
    pred3 = predict_by_prompting(zeroshot_model, tokenizer, encoder, example, few_shot_bank)
    votes = [pred1, pred2, pred3]
    return max(set(votes), key=votes.count)

# ------------------ Evaluate Ensemble ------------------
def run_ensemble_eval(val_data, train_data):
    gold = []
    pred = []
    bank = random.sample(train_data, 2)  # for few-shot prompting
    for ex in tqdm(val_data, desc="Ensemble Eval"):
        pred_idx = ensemble_predict(ex, bank)
        pred.append(LETTER_MAP[pred_idx])
        gold.append(LETTER_MAP[ex['target']])
    print("\n✅ Ensemble Accuracy:", accuracy_score(gold, pred))
    print(classification_report(gold, pred, labels=LETTER_MAP))

# Example usage:
# run_ensemble_eval(validation_data.to_dict(orient="records"), train_data.to_dict(orient="records"))


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
run_ensemble_eval(validation_data.to_dict(orient="records"), train_data.to_dict(orient="records"))

Ensemble Eval: 100%|██████████| 1272/1272 [10:14<00:00,  2.07it/s]


✅ Ensemble Accuracy: 0.2562893081761006
              precision    recall  f1-score   support

           A       0.25      0.80      0.39       330
           B       0.29      0.07      0.11       316
           C       0.32      0.07      0.11       352
           D       0.20      0.06      0.09       274

    accuracy                           0.26      1272
   macro avg       0.27      0.25      0.18      1272
weighted avg       0.27      0.26      0.18      1272

